In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
import pandas as pd
import numpy as np
import pprint
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

# Raw data

In [ ]:
train_pq_name = "otto_train_parquet"
test_pq_name = "otto_test_parquet"

In [ ]:
train_pq = context.catalog.load(train_pq_name)
test_pq = context.catalog.load(test_pq_name)

In [ ]:
print(train_pq.shape)
print(test_pq.shape)
train_pq.head()

In [ ]:
test_pq.head()

In [ ]:
grouped_df = train_pq.groupby("session").agg(
        length = ("session", "count"),
        mean_timestamp = ("ts", "mean"),
        event_types = ("type", "nunique"),
    )

In [ ]:
grouped_df.loc[:, "mean_timestamp"] = pd.qcut(grouped_df.loc[:, "mean_timestamp"], q=5, duplicates="drop").astype("category")

# Samples

In [ ]:
train_sample_name = "otto_train_preprocessed"
test_sample_name = "otto_test_preprocessed"

In [ ]:
train_sample = context.catalog.load(train_sample_name)
test_sample = context.catalog.load(test_sample_name)

In [ ]:
train_sample

# Saving testing fixture raw

In [ ]:
test_testing_sample = test_pq.sample(frac=0.001, random_state=42)

In [ ]:
test_testing_sample

In [ ]:
test_testing_sample.to_csv("../src/tests/fixtures/dataframes/otto_raw_sample.csv", index=False)

# Saving testing fixture preprocessed

In [ ]:
train_preprocessed_sample = train_sample.sample(frac=0.1, random_state=42)

In [ ]:
train_preprocessed_sample.shape

In [ ]:
train_preprocessed_sample.to_csv("../src/tests/fixtures/dataframes/otto_preprocessed_small_sample.csv", index=False)

In [ ]:
train_sample_name = "otto_train_preprocessed"

In [ ]:
hm_transaction = context.catalog.load("hm_to_act_train.transactions")

In [ ]:
hm_articles = hm_transaction.loc[:, ["article_id"]].drop_duplicates()
hm_articles.to_parquet("../data/03_primary/hm/hm_to_act_train_articles.parquet", index=False)

In [ ]:
hm_customers = hm_transaction.loc[:, ["customer_id"]].drop_duplicates()
hm_customers.to_parquet("../data/03_primary/hm/hm_to_act_train_customers.parquet", index=False)

In [ ]:
context.catalog.load("graph_recommendation_preprocessing_hm.transactions_mapped")